### Import Libraries
Here we will use standard data science libraries for python including scikit-learn, numpy, pandas, and matplotlib.

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from collections import OrderedDict
%matplotlib inline

### Provide your credentials to the runtime
This allows the colab access to underlying data stored in google cloud resources.

In [25]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Optional: Enable data table display
Colab includes the `google.colab.data_table` package the can be used to display large pandas dataframes as an interactive data table. It can be enabled with:

In [26]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


If you would prefer to return to the classic Pandas datafram display, you can disable this by running:


```
%unload_ext google.colab.data_table
```



### Use BigQuery via magics to retrieve Park Weather Data
The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

Load the query into a `Dataframe`

In [30]:
# Save output in a variable `weather_data`

%%bigquery weather_data --project sgcd-reportingengine-prj
SELECT
   PARK_ID
  ,case
    when lower(Forecast_Weather) like '%rain%' OR lower(Forecast_Weather) like '%shower%' OR lower(Forecast_Weather) like '%drizzle%' then 'Rain'
    when lower(Forecast_Weather) like '%partly%' OR lower(Forecast_Weather) like '%mostly%' then 'Partly Cloudy'
    when lower(Forecast_Weather) like '%cloud%' then 'Clouds'
    else 'Clear'
   end Forecast_Weather
  ,case
    when lower(Actual_Weather) like '%rain%' or lower(Actual_Weather) like '%shower%' OR lower(Actual_Weather) like '%drizzle%' then 'Rain'
    when lower(Actual_Weather) like '%partly%' OR lower(Actual_Weather) like '%mostly%' OR lower(Actual_Weather) like '%overcast%' then 'Partly Cloudy'
    when lower(Actual_Weather) like '%cloud%' then 'Clouds'
    else 'Clear'
  end Actual_Weather
  ,coalesce(Forecast_Attendance, 0) as Forecast_Attendance
  ,coalesce(Projected_Attendance, 0) as Projected_Attendance
  ,coalesce(Budget_Attendance, 0) as Budget_Attendance
  ,coalesce(Actual_Attendance, 0) as Actual_Attendance
 FROM `sgcd-reportingengine-prj.test.CA_DailyParkInfo`
 where length(coalesce(Forecast_Weather, '')) > 0

Query is running:   0%|          |

Downloading:   0%|          |

Display the `Dataframe` content.

In [31]:
# display the `DataFrame` content.
weather_data

,PARK_ID,Forecast_Weather,Actual_Weather,Forecast_Attendance,Projected_Attendance,Budget_Attendance,Actual_Attendance
0,1,Rain,Clear,3700,2900,4500,2502
1,1,Rain,Rain,1500,500,3100,444
2,1,Clear,Clear,2200,2900,2200,2967
3,1,Clear,Clear,10500,11800,9000,12228
4,1,Clear,Clear,22300,0,23200,23469
...,...,...,...,...,...,...,...
579,1,Clear,Clear,24500,23300,24800,18389
580,1,Clear,Clear,8500,7300,8200,7683
581,1,Partly Cloudy,Clear,11400,14600,11400,13762
582,1,Partly Cloudy,Clear,3100,3000,2900,3222


### Adjust text features
With the data now loaded, we can move on to adjusting text columns into categories. The following step will convert forecast and actual weather text values such as `Clear` or `Rain` to numeric equivalents, and then apply that over the `Dataframe`.

In [32]:
weather_category_dict = {'Clear':0, 'Partly Cloudy':1, 'Cloudy':2, 'Rain':3, 'None':np.nan}

weather_data.Forecast_Weather = pd.Categorical(weather_data.Forecast_Weather, categories=weather_category_dict)
weather_data['Forecast_Weather'] = weather_data.Forecast_Weather.cat.codes

weather_data.Actual_Weather = pd.Categorical(weather_data.Actual_Weather, categories=weather_category_dict)
weather_data['Actual_Weather'] = weather_data.Actual_Weather.cat.codes

weather_data

,PARK_ID,Forecast_Weather,Actual_Weather,Forecast_Attendance,Projected_Attendance,Budget_Attendance,Actual_Attendance
0,1,3,0,3700,2900,4500,2502
1,1,3,3,1500,500,3100,444
2,1,0,0,2200,2900,2200,2967
3,1,0,0,10500,11800,9000,12228
4,1,0,0,22300,0,23200,23469
...,...,...,...,...,...,...,...
579,1,0,0,24500,23300,24800,18389
580,1,0,0,8500,7300,8200,7683
581,1,1,0,11400,14600,11400,13762
582,1,1,0,3100,3000,2900,3222


### Shape the training data
We can now shape the appropriate input features and output values to prepare for training. You can uncomment one of the `feature_names`
declarations depending on which attendance projection you wish to analyze against.

Shape the input

In [33]:
feature_names = ["Forecast_Weather", "Projected_Attendance"]
#feature_names = ["Forecast_Weather", "Forecasted_Attendance"]
#feature_names = ["Forecast_Weather", "Budget_Attendance"]
attendance_output_name = "Actual_Attendance"

X = weather_data[feature_names]
X.head()

,Forecast_Weather,Projected_Attendance
0,3,2900
1,3,500
2,0,2900
3,0,11800
4,0,0


Shape the output

In [34]:
y_attendance = weather_data[attendance_output_name]
y_attendance.head()

0     2502
1      444
2     2967
3    12228
4    23469
Name: Actual_Attendance, dtype: Int64

### Split Training and Test Data
The next step is to properly split our training data into samples for training and verification or testing. The command below will select random samples of training data and reserve ~20% of the sample data for testing.